This notebook is a brief sketch of how to use the Deutsch-Jozsa algorithm.
We start by declaring all necessary imports.

In [ ]:
from itertools import product

from mock import patch

from grove.deutsch_jozsa.deutsch_jozsa import DeutschJosza

The Deutsch-Jozsa Algorithm can be used to determine if a binary-valued function is constant or balanced (returns 0 on exactly half of its inputs).

In [ ]:
bit_value = '0'
bit = ("0", "1")
constant_bitmap = {}

# We construct the bitmap for the algorithm
for bitstring in product(bit, repeat=2):
    bitstring = "".join(bitstring)
    constant_bitmap[bitstring] = bit_value

We verify that we have a constant bitmap below.

In [ ]:
for value in constant_bitmap.values():
    assert value == bit_value, "The constant_bitmap is not constant with value bit_value."

To use the Deutsch-Jozsa algorithm on a Quantum Hardware we need to define the connection to the QVM or QPU. However we don't have a real connection in this notebook, so we just mock out the response. If you run this notebook, ensure to replace `cxn` with a pyQuil connection object.

In [ ]:
with patch("pyquil.api.QuantumComputer") as qc:
    qc.run.return_value = [[0], [0]]

Now let's run the Deutsch Jozsa algorithm. We instantiate the Deutsch Jozsa object and then call its `is_constant` method with the connection object and the bitmap we defined above. Finally we assert its correctness by checking the output. (The method returns a boolean, so here we just check the returned boolean.)

In [ ]:
dj = DeutschJosza()
is_constant = dj.is_constant(qc, constant_bitmap)
assert is_constant, "The algorithm said the function was balanced."

If the assertion succeeded we know the algorithm returned the zero bitstring and successfully recognized the function as constant.